In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime
#turns seconds into a string format ffmpeg uses
def seconds_formatter(sec):
    x = "0" + str(datetime.timedelta(seconds=sec))[:-3]
    return x

## Random Forest Classifier
identify reaches, successes, failures

# Project Variables 
### Change as needed.

In [154]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\A_copiedDeepCut_resnet50_goprolensfrontJun25shuffle1_60002.h5")
#remove extra level "scorer"
df.columns = df.columns.droplevel()


frame_rate = 119.88

total_num_frames = df.shape[0]

cols = ['Nose', 'DomInside', 'Pellet', 'Index', 'OtherHand']

classes = ['n', 's', 'f', 'r']

frames_before = 12

frames_after = 30

step_size = 3

seed = 74
df.describe()

bodyparts          Nose                                    Pellet  \
coords                x             y    likelihood             x   
count      55026.000000  55026.000000  55026.000000  55026.000000   
mean        1330.184539    646.782603      0.376942   1379.689905   
std          201.531635    222.608088      0.389379    420.195539   
min            0.370821     -5.005973      0.000485      0.487646   
25%         1317.909296    564.589215      0.020343   1341.302102   
50%         1340.824574    687.884438      0.172433   1371.627043   
75%         1383.259692    789.351130      0.815848   1420.980582   
max         2524.008662   1520.601292      0.998947   2688.581522   

bodyparts                                 DomInside                \
coords                y    likelihood             x             y   
count      55026.000000  55026.000000  55026.000000  55026.000000   
mean         890.630344      0.353468   1348.746314    698.338131   
std          352.130231      0.381447    177.198718    190.104141   
min            0.977137      0.000128     -1.745432     -2.163946   
25%          679.328163      0.030228   1315.735032    609.551925   
50%          934.700097      0.138022   1355.566452    714.161687   
75%         1034.587177      0.825975   1411.890644    801.298333   
max         1525.692149      0.997842   2539.297546   1519.157574   

bodyparts                  DomOutside      ...             Middle  \
coords       likelihood             x      ...         likelihood   
count      55026.000000  55026.000000      ...       55026.000000   
mean           0.042437   1345.995294      ...           0.073249   
std            0.148373    184.583817      ...           0.170021   
min            0.000050     -3.968345      ...           0.000058   
25%            0.002453   1302.127836      ...           0.003987   
50%            0.005473   1339.641322      ...           0.011672   
75%            0.014076   1411.649381      ...           0.041728   
max            0.997374   2542.607008      ...           0.987304   

bodyparts          Ring                                     Pinky  \
coords                x             y    likelihood             x   
count      55026.000000  55026.000000  55026.000000  55026.000000   
mean        1347.171692    721.797350      0.066174   1346.244365   
std          204.993579    178.393742      0.158645    203.967791   
min           -4.533683     -1.481583      0.000048     -3.420242   
25%         1308.492969    652.538706      0.004266   1304.805376   
50%         1339.759142    721.430287      0.013019   1339.186867   
75%         1405.858760    814.640043      0.040963   1413.889349   
max         2701.087938   1517.565257      0.985866   2704.617106   

bodyparts                                 OtherHand                \
coords                y    likelihood             x             y   
count      55026.000000  55026.000000  55026.000000  55026.000000   
mean         726.047859      0.040276   1410.490640    692.802901   
std          180.706104      0.133038    157.962020    158.586212   
min           -2.252955      0.000062      1.162153     -1.754390   
25%          652.930016      0.002084   1357.557903    617.124485   
50%          728.146373      0.005336   1424.446016    677.042769   
75%          811.043618      0.015188   1465.638888    773.248826   
max         1518.772220      0.986586   2538.984463   1519.771770   

bodyparts                
coords       likelihood  
count      55026.000000  
mean           0.177920  
std            0.261808  
min            0.000121  
25%            0.012334  
50%            0.053886  
75%            0.212803  
max            0.997119  

[8 rows x 27 columns]

In [155]:
dff = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\A_copiedDeepCut_resnet50_goprolensfrontJun25shuffle1_60002.h5")
#remove extra level "scorer"
dff.columns = df.columns.droplevel()
dff.describe()

coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,...,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
count,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,...,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000,55026.000000
mean,1330.184539,646.782603,0.376942,1379.689905,890.630344,0.353468,1348.746314,698.338131,0.042437,1345.995294,...,0.073249,1347.171692,721.797350,0.066174,1346.244365,726.047859,0.040276,1410.490640,692.802901,0.177920
std,201.531635,222.608088,0.389379,420.195539,352.130231,0.381447,177.198718,190.104141,0.148373,184.583817,...,0.170021,204.993579,178.393742,0.158645,203.967791,180.706104,0.133038,157.962020,158.586212,0.261808
min,0.370821,-5.005973,0.000485,0.487646,0.977137,0.000128,-1.745432,-2.163946,0.000050,-3.968345,...,0.000058,-4.533683,-1.481583,0.000048,-3.420242,-2.252955,0.000062,1.162153,-1.754390,0.000121
25%,1317.909296,564.589215,0.020343,1341.302102,679.328163,0.030228,1315.735032,609.551925,0.002453,1302.127836,...,0.003987,1308.492969,652.538706,0.004266,1304.805376,652.930016,0.002084,1357.557903,617.124485,0.012334
50%,1340.824574,687.884438,0.172433,1371.627043,934.700097,0.138022,1355.566452,714.161687,0.005473,1339.641322,...,0.011672,1339.759142,721.430287,0.013019,1339.186867,728.146373,0.005336,1424.446016,677.042769,0.053886
75%,1383.259692,789.351130,0.815848,1420.980582,1034.587177,0.825975,1411.890644,801.298333,0.014076,1411.649381,...,0.041728,1405.858760,814.640043,0.040963,1413.889349,811.043618,0.015188,1465.638888,773.248826,0.212803
max,2524.008662,1520.601292,0.998947,2688.581522,1525.692149,0.997842,2539.297546,1519.157574,0.997374,2542.607008,...,0.987304,2701.087938,1517.565257,0.985866,2704.617106,1518.772220,0.986586,2538.984463,1519.771770,0.997119


In [156]:
def frame_to_time(frame):
    return round(frame/frame_rate, 3)
def time_to_frame(time):
    return int(round(time*frame_rate))

### Functions for Creating Classifier

In [157]:
import random
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

#Functions to get training data
def get_dataset(df, start_frames):
    print(df.shape)
    dataframe = df[cols]
    dataset = []
    for start_f in start_frames:
        temp_df = dataframe.iloc[start_f - frames_before: start_f + frames_after]
        if len(temp_df.values.flatten()) == 0:
            print(start_f)
            print(dataframe.shape)
        dataset.append(temp_df.values.flatten())
    return np.asarray(dataset)
def get_non_reach_frames(df, num_frames, blacklist, avoidance_radius):
    non_reaches = []
    random.seed(seed)
    while len(non_reaches) < num_frames:
        #hard code 3 non-reaches
        frame = random.randint(frames_before+10, df.shape[0] - frames_after)

        too_close = False
        for start_time in blacklist:
            if abs(frame - start_time) < avoidance_radius:
                print('no')
                too_close = True
                break
        if not too_close:
            non_reaches.append(frame)
    return non_reaches

#Use training features and labels to create Random Forest Classifier
def create_classifier(features, labels, n_trees, test_proportion=0.25):
    train_feats, test_feats, train_labs, test_labs = train_test_split(features, labels, test_size = test_proportion, random_state = seed)
    print('Training Features Shape:', train_feats.shape)
    print('Training Labels Shape:', train_labs.shape)
    print('Testing Features Shape:', test_feats.shape)
    print('Testing Labels Shape:', test_labs.shape)
    rf = RandomForestClassifier(n_estimators = n_trees, random_state = seed)
    # Train the model on training data
    print(train_feats.shape)
    print(train_labs.shape)
    rf.fit(train_feats, train_labs)
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_feats)
    print(predictions)
    print(test_labs)
    wrong_labels = []
#     for i in range(len(predictions)):
#         if predictions[i] != test_labs[i]:
#             frame = -1
#             for key in frame_to_features.keys():
#                 if (frame_to_features[key] == test_feats[i]).all():
#                     frame = key
#                     break
#             wrong_labels.append((frame_to_time(frame), predictions[i], test_labs[i]))
    for i in range(len(predictions)):
        if predictions[i] != test_labs[i]:
            frame = -1
            wrong_labels.append((predictions[i], test_labs[i]))
    print('Wrong Labels', wrong_labels)
    return rf, wrong_labels

### Training Data

In [158]:
print(list(zip(reach_times_injuredn2v3, reach_labels_injuredn2v3)))

[('21.196', 'f'), ('66.492', 's'), ('71.227', 'f'), ('88.349', 'f'), ('92.221', 'r'), ('99.692', 'f'), ('116.299', 'f'), ('127.703', 's'), ('135.332', 'f'), ('142.429', 'f'), ('147.11', 's'), ('151.609', 's'), ('157.722', 's'), ('163.4', 's'), ('172.798', 'f'), ('183.103', 'f'), ('204.054', 'f'), ('208.735', 'f'), ('216.329', 's')]


In [159]:
# all_frames = reach_framesA+non_reaches_framesA + reach_frames_n2v3+non_reaches_frames_n2v3 + reach_frames_n3v3+non_reaches_frames_n3v3 + reach_frames_injuredn2v3+non_reaches_frames_injuredn2v3
# vid_labels = ['A' for x in labelsA] + ['n2v3' for x in labels_n2v3] + ['n3v3' for x in labels_n3v3] + + ['injuredn2v3' for x in labels_injuredn2v3]
# frame_to_features = dict(zip(list(zip(all_frames, vid_labels)), compiled_features))

# print(len(all_frames))
#"C:\Users\vjj14\Documents\GoProLens\3lens\gp2.7k3lensA1.mp4"
df_A = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\gp2.7k3lensA1DeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
df_A.columns = df_A.columns.droplevel()
reach_timesA = [19.12, 23.752, 24.186, 24.630, 36.846, 43.652, 44.042, 46.964, 60.218, 62.897, 64.787, 72.045, 85.174, 88.196, 
           91.652, 93.248, 93.814, 106.294, 106.867, 111.344, 112.249, 113.407, 115.474, 115.918, 129.119, 136.837, 138.581]
reach_framesA = [time_to_frame(x) for x in reach_timesA]
reach_labelsA = ['r', 'f', 'r', 'r', 's', 'f', 's', 'r', 's', 'r', 'r', 'r', 's', 'r', 's', 'r', 'r', 'r', 'f', 'r', 'r', 
                      'r', 'f', 'r', 's', 'r', 'r']
non_reaches_framesA = get_non_reach_frames(df_A, len(reach_framesA)//2, reach_framesA, 12)
featuresA = get_dataset(df_A, reach_framesA+non_reaches_framesA)
labelsA = np.asarray(reach_labelsA + ['n' for x in non_reaches_framesA])


#"D:\DCIM\100GOPRO\2019.07.08_CC42970_N2_V3_2.7k_120FPS_Label_copy.mp4"
df_n2v3 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42970_N2_V3_2.7k_120FPS_Label_copyDeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
df_n2v3.columns = df_n2v3.columns.droplevel()
reach_times_n2v3 = [12.325, 16.568, 17.55, 18.912, 20.41, 22.1, 22.92, 24.699, 27.4, 29.754, 30.431, 31.39, 32.692, 37.731, 
           38.258, 42.86, 45.714, 56.45, 76.803, 77.86, 86.2, 86.75, 91.13, 91.649, 97.563, 98.1, 106.739, 107.292, 116.412, 
           121.49, 122.236, 124.775, 125.375, 134.324, 134.859, 142.37, 143.0, 150.0, 159.435, 160.04, 165.02, 
           165.633, 174.018, 177.129, 179.23, 183.719, 185.755, 192.041, 197.214, 198.66, 201.286, 203.34, 204.15, 206.253, 
           209.36, 213.027, 213.895, 217.676, 218.325, 218.982, 223.198, 226.56, 230.524, 233.61, 234.286, 235.838, 236.316, 
           237.091, 250.448, 251.206, 253.05]
reach_frames_n2v3 = [time_to_frame(x) for x in reach_times_n2v3]
reach_labels_n2v3 = ['s', 'r', 'f', 'r', 'r', 'f', 'r', 'r', 'r', 'f', 'r', 'r', 'r', 'f', 'r', 'r', 's', 'r', 'f', 'r', 
                  'f', 'r', 'f', 'r', 'f', 'r', 'f', 'r', 'f', 'r', 'r', 'f', 'r', 'f', 'r', 'f', 'r', 'f', 'f', 'r', 
                  'f', 'r', 's', 'r', 's', 'f', 'r', 's', 'f', 'r', 'f', 'r', 'r', 's', 'r', 'f', 'r', 'r', 'f', 'r', 's', 
                  'r', 's', 'r', 'r', 'f', 'f', 'r', 'f', 'r', 'r']
non_reaches_frames_n2v3 = get_non_reach_frames(df_n2v3, len(reach_frames_n2v3)//2, reach_frames_n2v3, 12)
features_n2v3 = get_dataset(df_n2v3, reach_frames_n2v3+non_reaches_frames_n2v3)
labels_n2v3 = np.asarray(reach_labels_n2v3 + ['n' for x in non_reaches_frames_n2v3])
print(len(labels_n2v3))
print(len(features_n2v3))
print(len(reach_frames_n2v3))
print(len(non_reaches_frames_n2v3))

#"D:\DCIM\100GOPRO\2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label_copy.mp4"
df_n3v3 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label_copyDeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
df_n3v3.columns = df_n3v3.columns.droplevel()
reach_times_n3v3 = ['11.589', '17.755', '18.474', '20.810', '22.457', '25.491', '37.403', '41.884', '51.558', '57.359', 
                         '64.503', '78.54', '84.611', '96.276', '108.619', '117.017', '123.131', '130.131', '157.557', 
                         '164.342', '207.559']
reach_frames_n3v3 = [time_to_frame(float(x)) for x in reach_times_n3v3]
reach_labels_n3v3 = ['s', 'f', 'r', 'r', 'r', 's', 'f', 's', 'f', 's', 'f', 'f', 's', 'f', 's', 's', 'f', 'f', 's', 
                          's', 'f']
non_reaches_times_n3v3 = [3.333, 126.886, 133.355, 205.494, 77.901, 94.947, 26.119, 73.896, 166.391, 51.277, 5.579, 16.697, 19.257, 22.71, 28.645, 31.2]
non_reaches_frames_n3v3 = [time_to_frame(x) for x in non_reaches_times_n3v3]
features_n3v3 = get_dataset(df_n3v3, reach_frames_n3v3+non_reaches_frames_n3v3)
labels_n3v3 = np.asarray(reach_labels_n3v3 + ['n' for x in non_reaches_frames_n3v3])

#"D:\DCIM\100GOPRO\2019.07.08_CC42973_N2INJURED_V3_2.7k_120FPS_Label.MP4"
df_injuredn2v3 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42973_N2INJURED_V3_2.7k_120FPS_Label_copyDeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
df_injuredn2v3.columns = df_injuredn2v3.columns.droplevel()
reach_times_injuredn2v3 = ['21.196', '66.492', '71.227', '88.349', '92.221', '99.692', '116.299', '127.703', '135.332', 
                                '142.429', '147.11', '151.609', '157.722', '163.4', '172.798', '183.103', '204.054', '208.735', '216.329']
reach_frames_injuredn2v3 = [time_to_frame(float(x)) for x in reach_times_injuredn2v3]
reach_labels_injuredn2v3 = ['f', 's', 'f', 'f', 'r', 'f', 'f', 's', 'f', 'f', 's', 's', 's', 's', 'f', 'f', 'f', 'f', 's']
non_reaches_times_injuredn2v3 = [157.463, 38.734, 62.743, 66.176, 88.856, 94.266, 171.907, 179.534, 184.588, 207.388, 72.36, 86.4, 88.5, 90.172]
non_reaches_frames_injuredn2v3 = [time_to_frame(x) for x in non_reaches_times_injuredn2v3]
features_injuredn2v3 = get_dataset(df_injuredn2v3, reach_frames_injuredn2v3+non_reaches_frames_injuredn2v3)
labels_injuredn2v3 = np.asarray(reach_labels_injuredn2v3 + ['n' for x in non_reaches_frames_injuredn2v3])

# frame_to_features

(17061, 27)
no
no
(30598, 27)
106
106
71
35
(25677, 27)
(26534, 27)


In [160]:
compiled_features = np.concatenate((featuresA, features_n2v3, features_n3v3, features_injuredn2v3))
compiled_labels = np.concatenate((labelsA, labels_n2v3, labels_n3v3, labels_injuredn2v3))
print(list(compiled_labels).count('r'))
print(list(compiled_labels).count('f'))
print(list(compiled_labels).count('s'))
print(list(compiled_labels).count('n'))

60
48
30
78


In [161]:
attempt_classifier = create_classifier(compiled_features, compiled_labels, 1000, test_proportion=0.01)[0]

Training Features Shape: (213, 630)
Training Labels Shape: (213,)
Testing Features Shape: (3, 630)
Testing Labels Shape: (3,)
(213, 630)
(213,)
['f' 'n' 'f']
['f' 'n' 's']
Wrong Labels [('f', 's')]


In [151]:
wrong_ones = []
for i in range(100, 200):
    print(i)
    seed = i
    wrong_ones.extend(create_classifier(compiled_features, compiled_labels, 1000, test_proportion=0.25)[1])
wrong_ones.sort(key= lambda x: x[0])
wrong_df = pd.DataFrame(wrong_ones, columns = ['guess', 'actual'])
wrong_df.groupby(wrong_df.columns.tolist(),as_index=False).size()

100
Training Features Shape: (162, 630)
Training Labels Shape: (162,)
Testing Features Shape: (54, 630)
Testing Labels Shape: (54,)
(162, 630)
(162,)
['r' 'n' 'n' 'n' 'n' 'r' 'f' 'n' 'f' 'r' 'r' 'n' 'n' 'n' 'r' 'n' 's' 's'
 'n' 'f' 'n' 's' 'n' 'f' 'n' 'n' 'n' 'r' 'n' 'f' 'n' 'n' 'r' 's' 'f' 'r'
 'r' 'n' 'n' 'f' 'r' 'r' 'n' 'n' 'n' 's' 'n' 'r' 'r' 'f' 'n' 'n' 'r' 'r']
['r' 'f' 'n' 'n' 'n' 'r' 'f' 'n' 'f' 'r' 'r' 'n' 'n' 'n' 'r' 'n' 's' 's'
 'n' 'f' 'n' 'f' 'n' 'r' 'n' 'n' 'n' 'f' 'n' 'r' 'r' 'n' 'r' 's' 'f' 'r'
 'r' 'n' 'n' 'f' 'r' 'r' 'n' 'n' 'n' 's' 'n' 'r' 'r' 'f' 'n' 'n' 'r' 'r']
Wrong Labels [('n', 'f'), ('s', 'f'), ('f', 'r'), ('r', 'f'), ('f', 'r'), ('n', 'r')]
101
Training Features Shape: (162, 630)
Training Labels Shape: (162,)
Testing Features Shape: (54, 630)
Testing Labels Shape: (54,)
(162, 630)
(162,)
['s' 'f' 'n' 'r' 'r' 'f' 'f' 'r' 'n' 'f' 's' 'n' 'f' 'n' 's' 'n' 'f' 'r'
 'r' 'f' 'n' 'r' 'n' 'r' 'n' 'r' 'f' 'n' 'n' 'r' 'n' 's' 'f' 'n' 'r' 'n'
 'r' 'r' 'n' 'n' 'n' 'f' 'n'

['f' 'n' 'n' 'f' 's' 'f' 'r' 'n' 'f' 's' 'n' 'r' 'n' 's' 'r' 'n' 'n' 'f'
 'r' 'f' 's' 'r' 's' 'r' 'f' 'n' 'n' 'n' 'f' 'f' 's' 'n' 'r' 'f' 'n' 'n'
 'r' 'n' 'n' 'r' 'r' 'n' 'f' 'f' 'r' 'n' 's' 'n' 'f' 'f' 'n' 'n' 'f' 'n']
['f' 'n' 'n' 'f' 's' 'f' 'r' 'n' 's' 's' 'n' 'r' 'n' 's' 'r' 'n' 'n' 'f'
 'r' 'f' 's' 'r' 's' 'r' 'f' 'n' 'n' 'n' 'f' 'r' 's' 'n' 'r' 'f' 'n' 'n'
 'r' 'n' 'r' 'r' 'r' 'n' 'f' 'f' 'n' 'n' 's' 'n' 'f' 'f' 'n' 'n' 's' 'r']
Wrong Labels [('f', 's'), ('f', 'r'), ('n', 'r'), ('r', 'n'), ('f', 's'), ('n', 'r')]
113
Training Features Shape: (162, 630)
Training Labels Shape: (162,)
Testing Features Shape: (54, 630)
Testing Labels Shape: (54,)
(162, 630)
(162,)
['n' 'r' 'f' 'f' 'n' 'n' 'f' 'f' 'f' 'f' 'n' 's' 'n' 'r' 'n' 'f' 'n' 'r'
 'r' 'r' 'f' 'f' 'f' 'r' 'r' 'n' 'r' 'r' 'n' 'n' 'n' 's' 'n' 'n' 'n' 'r'
 'r' 'f' 'n' 'f' 'n' 'f' 'n' 'n' 'n' 'f' 's' 'r' 's' 'f' 'r' 'f' 's' 'n']
['n' 'r' 'f' 'f' 'n' 'n' 's' 'f' 'f' 's' 'n' 's' 'n' 'r' 'n' 'f' 'n' 'r'
 'r' 'r' 's' 'f' 's' 'r' 'n' 'n

KeyboardInterrupt: 

# Find Success, Fails, and Reaches

In [166]:
from statistics import mode, StatisticsError
#Helper Functions
def get_predicted_times(classifier, df):
    dataframe = df[cols]
    inputs = []
    num_frames = frames_before + frames_after
    for index in range(num_frames, dataframe.shape[0] - num_frames, step_size):
        temp_df = dataframe.iloc[index - frames_before: index + frames_after]
        inputs.append(temp_df.values.flatten())
    inputs = np.asarray(inputs)
    predictions = classifier.predict(inputs)
    return predictions, list(range(num_frames, dataframe.shape[0] - num_frames, step_size))
def get_starts(lst):

    def most_common(l):
        try:
            return [mode(l)]
        except StatisticsError as e:
            # will only return the first element if no unique mode found
            if 'no unique mode' in e.args[0]:
                print("TIE: ", l)
                return [l[0]]
            # this is for "StatisticsError: no mode for empty data"
            # after calling mode([])
        raise
    lst = [(frame_to_time(x[0]), x[1]) for x in lst if x[1] != 'n']
    print(lst[0])
    lst.insert(0, (-999, 'sentinel'))
    starts = []
    run = 0
    classes_list = []
    for i in range(0, len(lst)):
        run += 1
        if lst[i][1] != 'sentinel':
            classes_list.append(lst[i][1])
        if lst[i][0] - lst[i-1][0] > 0.2:
            starts.append((lst[i][0], most_common(classes_list), classes_list))
            classes_list = []
            run = 0
    return starts

#Find Reaches, 
def get_classified_starts(classifier, df):
    predictions2, start_frames = get_predicted_times(classifier, df)
    class_to_predictions = {}
    return get_starts(list(zip(start_frames, predictions2)))
#     for start, prediction in zip(start_frames, predictions2):
#         for c in classes:
#             if prediction == c:
#                 if c in class_to_predictions.keys():
#                     class_to_predictions[c].append(frame_to_time(start))
#                 else:
#                     class_to_predictions[c] = [frame_to_time(start)]
#     for c in classes:
#         hold = get_starts(class_to_predictions[c])
#         class_to_predictions[c] = hold
#     return class_to_predictions


In [167]:
test_df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42970_N3_V2_2.7k_120FPS_analyze_trimDeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
test_df.columns = test_df.columns.droplevel()
starts = get_classified_starts(attempt_classifier, test_df)
print(starts)
print(len(starts))
predicted_fails = [x[0] for x in starts if 'f' in x[1]]
predicted_reaches = [x[0] for x in starts if 'r' in x[1]]
predicted_successes = [x[0] for x in starts if 's' in x[1]]
print(predicted_fails)
print(predicted_reaches)
print(predicted_successes)

(2.002, 'r')
TIE:  ['r', 'f']
TIE:  ['r', 'r', 'r', 'r', 'r', 'r', 'f', 'f', 'f', 'f', 'f', 'f']
[(2.002, ['r'], ['r']), (2.878, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r']), (3.829, ['r'], ['r', 'f']), (4.605, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'r']), (6.006, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r']), (10.561, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'f', 'f', 'f', 'f', 'f', 'f']), (11.011, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'f']), (15.09, ['f'], ['f', 'f', 'f', 'f', 'f', 'r']), (15.666, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r']), (17.868, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r']), (18.594, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'f']), (19.595, ['f'], ['f', 'f', 'f', 's', 'r']), (20.946, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r']), (26.051, ['f'], ['r', 'f', 'f']), (26.777, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'r']), (28.328, ['r'], ['r', 'r']), (30.606, ['r'], ['r

In [49]:
test_labels = ['r', 'f', 'r', 'r', 'f', 'f', 'r', 'r', 'r', 'f', 'r', 'f', 'r', 's', 'r', 'r', 'r', 'r', 'r', 'r', 'f', 'r', 
               'r', 'r', 's', 'r', 'r', 'r', 's', 'r']
test_times = [2.08, 3.9, 4.7, 6.09, 10.66, 11.1, 15.1, 15.77, 17.9, 18.6, 19.6, 26, 28.4, 30.6, 37.2, 39.9, 41.8, 43.5, 45.7, 
              46.67, 48.4, 49, 49.25, 51.566, 52.7, 58.7, 60.5, 60.9, 64.15, 69.1]
print(list(zip(test_times, test_labels)))
print(test_labels.count('f'))
print(test_labels.count('r'))
print(test_labels.count('s'))

[(2.08, 'r'), (3.9, 'f'), (4.7, 'r'), (6.09, 'r'), (10.66, 'f'), (11.1, 'f'), (15.1, 'r'), (15.77, 'r'), (17.9, 'r'), (18.6, 'f'), (19.6, 'r'), (26, 'f'), (28.4, 'r'), (30.6, 's'), (37.2, 'r'), (39.9, 'r'), (41.8, 'r'), (43.5, 'r'), (45.7, 'r'), (46.67, 'r'), (48.4, 'f'), (49, 'r'), (49.25, 'r'), (51.566, 'r'), (52.7, 's'), (58.7, 'r'), (60.5, 'r'), (60.9, 'r'), (64.15, 's'), (69.1, 'r')]
6
21
3


In [78]:
x = sorted(predicted_fails + predicted_reaches + predicted_successes)
print(x)
print(len(x))
import numpy as np
np.savetxt("file_name.csv", x, delimiter=",", fmt='%s', header='times')

[2.002, 3.854, 4.63, 6.006, 10.586, 11.036, 15.09, 15.691, 17.868, 18.594, 19.595, 20.971, 26.051, 28.328, 30.606, 37.187, 38.263, 39.79, 41.742, 43.493, 45.671, 46.597, 48.398, 48.949, 50.325, 51.502, 52.803, 58.659, 60.41, 60.911, 64.064, 68.118, 69.244]
33


In [169]:
test_df2 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42970_N2_V2_2.7k_120FPS_copyDeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
test_df2.columns = test_df2.columns.droplevel()
starts = get_classified_starts(attempt_classifier, test_df2)
print(starts)
print(len(starts))
predicted_fails = [x[0] for x in starts if 'f' in x[1]]
predicted_reaches = [x[0] for x in starts if 'r' in x[1]]
predicted_successes = [x[0] for x in starts if 's' in x[1]]
print(predicted_fails)
print(predicted_reaches)
print(predicted_successes)

(14.64, 'f')
TIE:  ['f', 'f', 'f', 's', 's', 's', 's', 'f']
TIE:  ['f', 'f', 'f', 'f', 's', 's', 's', 's', 'r']
[(14.64, ['f'], ['f']), (32.633, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']), (42.017, ['f'], ['f', 'f', 'f', 's', 's', 's', 's', 'f']), (42.568, ['f'], ['f', 'f', 'f', 'f', 'f', 'f']), (50.901, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'f', 's', 's', 'r']), (53.403, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'f']), (53.954, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']), (54.68, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']), (55.33, ['f'], ['f', 'f', 'f', 'f']), (55.706, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']), (57.157, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']), (58.158, ['f'], ['f', 'f', 'f', 'f', 'f', 'r']), (62.262, ['r'], ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'f']), (62.863, ['f'], ['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']), (64.064, ['f']

In [124]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
import xgboost
def create_classifierXGB(features, labels, n_trees, test_proportion=0.25):
    train_feats, test_feats, train_labs, test_labs = train_test_split(features, labels, test_size = test_proportion, random_state = seed)
    print('Training Features Shape:', train_feats.shape)
    print('Training Labels Shape:', train_labs.shape)
    print('Testing Features Shape:', test_feats.shape)
    print('Testing Labels Shape:', test_labs.shape)
    rf = xgboost.XGBClassifier()
    # Train the model on training data
    print(train_feats.shape)
    print(train_labs.shape)
    rf.fit(train_feats, train_labs)
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_feats)
    print(predictions)
    print(test_labs)
    wrong_labels = []
#     for i in range(len(predictions)):
#         if predictions[i] != test_labs[i]:
#             frame = -1
#             for key in frame_to_features.keys():
#                 if (frame_to_features[key] == test_feats[i]).all():
#                     frame = key
#                     break
#             wrong_labels.append((frame_to_time(frame), predictions[i], test_labs[i]))
    for i in range(len(predictions)):
        if predictions[i] != test_labs[i]:
            frame = -1
            wrong_labels.append((predictions[i], test_labs[i]))
    print('Wrong Labels', wrong_labels)
    return rf, wrong_labels
def create_classifierSVC(features, labels, n_trees, test_proportion=0.25):
    train_feats, test_feats, train_labs, test_labs = train_test_split(features, labels, test_size = test_proportion, random_state = seed)
    print('Training Features Shape:', train_feats.shape)
    print('Training Labels Shape:', train_labs.shape)
    print('Testing Features Shape:', test_feats.shape)
    print('Testing Labels Shape:', test_labs.shape)
    rf = svm.SVC()
    # Train the model on training data
    print(train_feats.shape)
    print(train_labs.shape)
    rf.fit(train_feats, train_labs)
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_feats)
    print(predictions)
    print(test_labs)
    wrong_labels = []
#     for i in range(len(predictions)):
#         if predictions[i] != test_labs[i]:
#             frame = -1
#             for key in frame_to_features.keys():
#                 if (frame_to_features[key] == test_feats[i]).all():
#                     frame = key
#                     break
#             wrong_labels.append((frame_to_time(frame), predictions[i], test_labs[i]))
    for i in range(len(predictions)):
        if predictions[i] != test_labs[i]:
            frame = -1
            wrong_labels.append((predictions[i], test_labs[i]))
    print('Wrong Labels', wrong_labels)
    return rf, wrong_labels
def create_classifier_tree(features, labels, n_trees, test_proportion=0.25):
    train_feats, test_feats, train_labs, test_labs = train_test_split(features, labels, test_size = test_proportion, random_state = seed)
    print('Training Features Shape:', train_feats.shape)
    print('Training Labels Shape:', train_labs.shape)
    print('Testing Features Shape:', test_feats.shape)
    print('Testing Labels Shape:', test_labs.shape)
    rf = tree.DecisionTreeClassifier()
    # Train the model on training data
    print(train_feats.shape)
    print(train_labs.shape)
    rf.fit(train_feats, train_labs)
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_feats)
    print(predictions)
    print(test_labs)
    wrong_labels = []
#     for i in range(len(predictions)):
#         if predictions[i] != test_labs[i]:
#             frame = -1
#             for key in frame_to_features.keys():
#                 if (frame_to_features[key] == test_feats[i]).all():
#                     frame = key
#                     break
#             wrong_labels.append((frame_to_time(frame), predictions[i], test_labs[i]))
    for i in range(len(predictions)):
        if predictions[i] != test_labs[i]:
            frame = -1
            wrong_labels.append((predictions[i], test_labs[i]))
    print('Wrong Labels', wrong_labels)
    return rf, wrong_labels

In [123]:
forest_classifier = create_classifier(compiled_features, compiled_labels, 1000, test_proportion=0.25)[0]

Training Features Shape: (162, 1728)
Training Labels Shape: (162,)
Testing Features Shape: (54, 1728)
Testing Labels Shape: (54,)
(162, 1728)
(162,)
['f' 's' 's' 'r' 'r' 'n' 'f' 'n' 's' 'r' 'n' 'f' 'r' 'f' 'n' 'n' 'r' 's'
 'n' 'r' 'n' 's' 'r' 'f' 'r' 'r' 'n' 'r' 's' 'r' 's' 'f' 'f' 'n' 's' 'n'
 'n' 'f' 'r' 'n' 'f' 'r' 's' 'n' 'f' 'f' 'n' 'n' 'n' 'n' 'r' 'r' 'n' 's']
['f' 's' 's' 'r' 'r' 'n' 'f' 'n' 's' 'r' 'n' 'n' 'r' 'f' 'n' 'n' 'r' 's'
 'n' 'r' 'n' 's' 'n' 'f' 'r' 'r' 'n' 'r' 's' 'r' 's' 'f' 'f' 'f' 's' 'n'
 'n' 'f' 'r' 'n' 'f' 'r' 's' 'n' 'r' 'f' 'n' 'n' 'n' 'n' 'f' 'r' 'f' 's']
Wrong Labels [('f', 'n'), ('r', 'n'), ('n', 'f'), ('f', 'r'), ('r', 'f'), ('n', 'f')]


In [121]:
XGB_classifier = create_classifierXGB(compiled_features, compiled_labels, 1000, test_proportion=0.25)[0]

Training Features Shape: (162, 1728)
Training Labels Shape: (162,)
Testing Features Shape: (54, 1728)
Testing Labels Shape: (54,)
(162, 1728)
(162,)
['f' 'f' 's' 'r' 'r' 'n' 'f' 'n' 's' 'r' 'n' 'n' 'r' 'f' 'n' 'n' 'r' 'f'
 'f' 'r' 'n' 'f' 'r' 'f' 'r' 'r' 'n' 'r' 's' 'r' 's' 'f' 'f' 'n' 's' 'n'
 'n' 'f' 'r' 'n' 'f' 'r' 's' 'n' 'f' 's' 'n' 'n' 'n' 'n' 'r' 'r' 'n' 's']
['f' 's' 's' 'r' 'r' 'n' 'f' 'n' 's' 'r' 'n' 'n' 'r' 'f' 'n' 'n' 'r' 's'
 'n' 'r' 'n' 's' 'n' 'f' 'r' 'r' 'n' 'r' 's' 'r' 's' 'f' 'f' 'f' 's' 'n'
 'n' 'f' 'r' 'n' 'f' 'r' 's' 'n' 'r' 'f' 'n' 'n' 'n' 'n' 'f' 'r' 'f' 's']
Wrong Labels [('f', 's'), ('f', 's'), ('f', 'n'), ('f', 's'), ('r', 'n'), ('n', 'f'), ('f', 'r'), ('s', 'f'), ('r', 'f'), ('n', 'f')]


C:\Users\vjj14\.conda\envs\dlc-windowsGPU\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [125]:
SVC_classifier = create_classifierSVC(compiled_features, compiled_labels, 1000, test_proportion=0.25)[0]

Training Features Shape: (162, 1728)
Training Labels Shape: (162,)
Testing Features Shape: (54, 1728)
Testing Labels Shape: (54,)
(162, 1728)
(162,)
['n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n'
 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n'
 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n']
['f' 's' 's' 'r' 'r' 'n' 'f' 'n' 's' 'r' 'n' 'n' 'r' 'f' 'n' 'n' 'r' 's'
 'n' 'r' 'n' 's' 'n' 'f' 'r' 'r' 'n' 'r' 's' 'r' 's' 'f' 'f' 'f' 's' 'n'
 'n' 'f' 'r' 'n' 'f' 'r' 's' 'n' 'r' 'f' 'n' 'n' 'n' 'n' 'f' 'r' 'f' 's']
Wrong Labels [('n', 'f'), ('n', 's'), ('n', 's'), ('n', 'r'), ('n', 'r'), ('n', 'f'), ('n', 's'), ('n', 'r'), ('n', 'r'), ('n', 'f'), ('n', 'r'), ('n', 's'), ('n', 'r'), ('n', 's'), ('n', 'f'), ('n', 'r'), ('n', 'r'), ('n', 'r'), ('n', 's'), ('n', 'r'), ('n', 's'), ('n', 'f'), ('n', 'f'), ('n', 'f'), ('n', 's'), ('n', 'f'), ('n', 'r'), ('n', 'f'), ('n', 'r'), ('n', 's'), ('n', 'r'), ('n', 'f'), ('n', 'f'), (